# 03 — Entraînement des modèles binaires

Entraîne 10 classifiers en formulation binaire :
- **HomeWin vs Not-HomeWin**
- **AwayWin vs Not-AwayWin**

Avec courbes ROC et split temporel (train < 2022, test ≥ 2022).

**Pré-requis :** avoir généré `data/all_leagues_combined.csv` avec le notebook `01_Data_Processing.ipynb`.

---
### Google Colab : configuration du chemin Drive

In [ ]:
# ============================================================
# CONFIGURATION — À modifier si nécessaire
# ============================================================
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/FootWork'
LOCAL_PROJECT_PATH = '..'

In [ ]:
# ============================================================
# Détection environnement + montage Drive
# ============================================================
import os, sys

try:
    import google.colab
    ON_COLAB = True
except ImportError:
    ON_COLAB = False

if ON_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_ROOT = DRIVE_PROJECT_PATH
else:
    PROJECT_ROOT = os.path.abspath(LOCAL_PROJECT_PATH)

os.chdir(PROJECT_ROOT)
sys.path.insert(0, PROJECT_ROOT)
print(f'Environnement : {"Google Colab" if ON_COLAB else "Local"}')
print(f'Répertoire de travail : {os.getcwd()}')

In [ ]:
if ON_COLAB:
    %pip install -q pyyaml xgboost lightgbm catboost scikit-learn seaborn tqdm

In [ ]:
# ============================================================
# Imports
# ============================================================
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (confusion_matrix, classification_report, accuracy_score,
                              balanced_accuracy_score, matthews_corrcoef, cohen_kappa_score,
                              roc_curve, auc)
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.calibration import CalibratedClassifierCV, CalibrationDisplay
from sklearn.frozen import FrozenEstimator
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
                               AdaBoostClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from src.Config.Config_Manager import ConfigManager
from src.Models.threshold_optimizer import find_optimal_threshold, predict_binary_with_threshold

config = ConfigManager('src/Config/configBT_1.yaml')
print('Config chargée.')

---
## Chargement et split temporel des données

In [ ]:
input_path = config.get_paths()['full_dataset']
exclude_columns = config.get_config_value('excluded_columns', default=[])
split_config = config.get_config_value('data_split', default={})

df = pd.read_csv(input_path)

# Targets binaires
df['home_win'] = (df['target_result'] == 'HomeWin').astype(int)
df['away_win'] = (df['target_result'] == 'AwayWin').astype(int)

print(f'Dataset : {len(df)} matchs × {len(df.columns)} colonnes')
print(f'HomeWin : {df["home_win"].sum()} ({df["home_win"].mean():.1%})')
print(f'AwayWin : {df["away_win"].sum()} ({df["away_win"].mean():.1%})')

In [ ]:
# Split temporel :
#   train  : year < 2020
#   cal    : 2020 <= year < 2022  (calibration + optimisation de seuil)
#   test   : year >= 2022
temporal_split_year = split_config.get('temporal_split_year', 2022)
cal_split_year      = split_config.get('cal_split_year', 2020)
cols_to_drop = [c for c in exclude_columns + ['target_result', 'home_win', 'away_win']
                if c in df.columns]

if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    train_df = df[df['date'].dt.year < cal_split_year].copy()
    cal_df   = df[(df['date'].dt.year >= cal_split_year) &
                  (df['date'].dt.year < temporal_split_year)].copy()
    test_df  = df[df['date'].dt.year >= temporal_split_year].copy()
    print(f'Split temporel :')
    print(f'  train  < {cal_split_year}        → {len(train_df)} matchs')
    print(f'  cal    {cal_split_year}–{temporal_split_year-1}      → {len(cal_df)} matchs')
    print(f'  test  >= {temporal_split_year}       → {len(test_df)} matchs')
    X_train = train_df.drop(cols_to_drop, axis=1, errors='ignore')
    X_cal   = cal_df.drop(cols_to_drop, axis=1, errors='ignore')
    X_test  = test_df.drop(cols_to_drop, axis=1, errors='ignore')
    y_home_train, y_home_cal, y_home_test = (
        train_df['home_win'], cal_df['home_win'], test_df['home_win']
    )
    y_away_train, y_away_cal, y_away_test = (
        train_df['away_win'], cal_df['away_win'], test_df['away_win']
    )
else:
    random_params = {k: v for k, v in split_config.items() if k != 'temporal_split_year'}
    X = df.drop(cols_to_drop, axis=1, errors='ignore')
    X_train, X_test, y_home_train, y_home_test, y_away_train, y_away_test = train_test_split(
        X, df['home_win'], df['away_win'], **random_params
    )
    X_cal, y_home_cal, y_away_cal = None, None, None

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_cal_scaled   = scaler.transform(X_cal) if X_cal is not None else None
X_test_scaled  = scaler.transform(X_test)

print(f'\nFeatures utilisées : {X_train.shape[1]}')

---
## Définition des modèles

In [ ]:
model_params = config.get_config_value('model_parameters', default={})

catboost_dir = Path('src/Models/Binary_Target/catboost_info')
catboost_dir.mkdir(parents=True, exist_ok=True)
catboost_p = {**model_params.get('catboost', {}), 'train_dir': str(catboost_dir)}

def make_models():
    return {
        'Random Forest':       RandomForestClassifier(**model_params.get('random_forest', {})),
        'Logistic Regression': LogisticRegression(**model_params.get('logistic_regression', {})),
        'SVM':                 SVC(**model_params.get('svm', {})),
        'Gradient Boosting':   GradientBoostingClassifier(**model_params.get('gradient_boosting', {})),
        'XGBoost':             XGBClassifier(**model_params.get('xgboost', {})),
        'LightGBM':            LGBMClassifier(**model_params.get('lightgbm', {})),
        'CatBoost':            CatBoostClassifier(**catboost_p),
        'KNN':                 KNeighborsClassifier(**model_params.get('knn', {})),
        'AdaBoost':            AdaBoostClassifier(**model_params.get('adaboost', {})),
        'Extra Trees':         ExtraTreesClassifier(**model_params.get('extra_trees', {})),
    }

print('Modèles définis.')

In [ ]:
# ============================================================
# Helper : entraînement avec suivi des courbes de loss
# ============================================================
def _fit_with_tracking(model, X_train, X_test, y_train, y_test, sample_weight=None):
    """Entraîne et capture les métriques par itération pour les boosters.

    sample_weight est propagé aux modèles qui n'acceptent pas class_weight
    dans leur constructeur (XGBoost, GradientBoosting, AdaBoost).
    """
    cls = model.__class__.__name__

    if cls == 'XGBClassifier':
        model.fit(X_train, y_train,
                  sample_weight=sample_weight,
                  eval_set=[(X_train, y_train), (X_test, y_test)],
                  verbose=False)
        evals = model.evals_result()
        mk = list(evals['validation_0'].keys())[0]
        return model, {'train': evals['validation_0'][mk],
                       'val':   evals['validation_1'][mk], 'metric': mk}

    elif cls == 'LGBMClassifier':
        evals_result = {}
        model.fit(X_train, y_train,
                  eval_set=[(X_train, y_train), (X_test, y_test)],
                  eval_names=['train', 'val'],
                  callbacks=[lgb.record_evaluation(evals_result)])
        mk = list(evals_result['train'].keys())[0]
        return model, {'train': evals_result['train'][mk],
                       'val':   evals_result['val'][mk], 'metric': mk}

    elif cls == 'CatBoostClassifier':
        base_dir = model.get_param('train_dir') or 'catboost_info'
        safe_dir = os.path.join(base_dir, f'run_{id(model)}')
        os.makedirs(safe_dir, exist_ok=True)
        model.set_params(train_dir=safe_dir)
        model.fit(X_train, y_train, eval_set=(X_test, y_test))
        evals = model.get_evals_result()
        lk = 'learn'      if 'learn'      in evals else list(evals.keys())[0]
        vk = 'validation' if 'validation' in evals else list(evals.keys())[-1]
        mk = list(evals[lk].keys())[0]
        return model, {'train': evals[lk][mk], 'val': evals[vk][mk], 'metric': mk}

    elif cls == 'GradientBoostingClassifier':
        model.fit(X_train, y_train, sample_weight=sample_weight)
        t = [accuracy_score(y_train, p) for p in model.staged_predict(X_train)]
        v = [accuracy_score(y_test,  p) for p in model.staged_predict(X_test)]
        return model, {'train': t, 'val': v, 'metric': 'accuracy'}

    elif cls == 'AdaBoostClassifier':
        model.fit(X_train, y_train, sample_weight=sample_weight)
        return model, None

    else:
        model.fit(X_train, y_train)
        return model, None


def _train_binary(models_dict, X_train, X_test, X_cal, y_train, y_test, y_cal,
                  target_name, label_names, output_dir):
    """Boucle d'entraînement commune pour HomeWin et AwayWin."""
    sw = compute_sample_weight('balanced', y_train)
    results = {}

    pbar = tqdm(models_dict.items(), total=len(models_dict), desc=f"{target_name} models")
    for name, model in pbar:
        pbar.set_description(f"{target_name}  {name:<22}")
        try:
            # CatBoost : class_weights dynamiques selon le déséquilibre réel
            if model.__class__.__name__ == 'CatBoostClassifier':
                cw = compute_class_weight('balanced', classes=np.array([0, 1]),
                                          y=np.array(y_train))
                model.set_params(class_weights=list(cw))

            model, loss_history = _fit_with_tracking(
                model, X_train, X_test, y_train, y_test, sample_weight=sw
            )
            y_pred = model.predict(X_test)

            acc     = accuracy_score(y_test, y_pred)
            bal_acc = balanced_accuracy_score(y_test, y_pred)
            mcc     = matthews_corrcoef(y_test, y_pred)
            kappa   = cohen_kappa_score(y_test, y_pred)
            cm      = confusion_matrix(y_test, y_pred)
            pbar.set_postfix(acc=f"{acc:.4f}", mcc=f"{mcc:.4f}")

            # Calibration + seuil optimisé (séquentiel — exceptions visibles)
            calibrated_model, calibrated_roc = None, None
            opt_thr, acc_opt, bal_opt, report_opt = None, None, None, None

            if X_cal is not None and hasattr(model, 'predict_proba'):
                try:
                    calibrated_model = CalibratedClassifierCV(FrozenEstimator(model), method='isotonic')
                    calibrated_model.fit(X_cal, y_cal)
                    cal_probs_test = calibrated_model.predict_proba(X_test)[:, 1]
                    fpr, tpr, _ = roc_curve(y_test, cal_probs_test)
                    calibrated_roc = auc(fpr, tpr)

                    # Courbe de calibration
                    fig, ax = plt.subplots(figsize=(5, 4))
                    raw_prob = model.predict_proba(X_cal)[:, 1]
                    cal_prob = calibrated_model.predict_proba(X_cal)[:, 1]
                    CalibrationDisplay.from_predictions(y_cal, raw_prob, n_bins=10, ax=ax,
                                                        label='Uncalibrated', color='steelblue')
                    CalibrationDisplay.from_predictions(y_cal, cal_prob, n_bins=10, ax=ax,
                                                        label='Calibrated', color='darkorange')
                    ax.set_title(f'Calibration — {name} ({target_name})')
                    ax.legend(fontsize=8)
                    fig.tight_layout()
                    fig.savefig(output_dir / f'calibration_{target_name.replace(" ","_")}_{name.replace(" ","_")}.png', dpi=150)
                    plt.show()

                    # Seuil optimal
                    cal_probs_cal = calibrated_model.predict_proba(X_cal)[:, 1]
                    opt_thr_val, opt_f1 = find_optimal_threshold(cal_probs_cal, np.array(y_cal))
                    y_pred_opt = predict_binary_with_threshold(cal_probs_test, opt_thr_val)
                    opt_thr    = (opt_thr_val, opt_f1)
                    acc_opt    = accuracy_score(y_test, y_pred_opt)
                    bal_opt    = balanced_accuracy_score(y_test, y_pred_opt)
                    report_opt = classification_report(y_test, y_pred_opt, target_names=label_names)

                    print(f'  {name}: cal_auc={calibrated_roc:.4f}  acc_opt={acc_opt:.4f}')
                except Exception as e:
                    import traceback
                    print(f'  [Calibration FAILED for {name}]:')
                    traceback.print_exc()

            # Matrice de confusion
            fig, ax = plt.subplots(figsize=(5, 4))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                        xticklabels=label_names, yticklabels=label_names, ax=ax)
            ax.set_title(f'{name} ({target_name})  Acc={acc:.3f}  MCC={mcc:.3f}')
            ax.set_ylabel('Vrai'); ax.set_xlabel('Prédit')
            fig.tight_layout()
            fig.savefig(output_dir / f'cm_{target_name.replace(" ","_")}_{name.replace(" ","_")}.png', dpi=150)
            plt.show()

            if loss_history:
                lh = loss_history
                _LOSS_METRICS = {'loss', 'logloss', 'multiclass', 'merror', 'mae', 'mse', 'rmse'}
                is_loss = any(tok in lh['metric'].lower() for tok in _LOSS_METRICS)
                best = int(np.argmin(lh['val']) if is_loss else np.argmax(lh['val']))
                fig, ax = plt.subplots(figsize=(7, 4))
                iters = range(1, len(lh['train']) + 1)
                ax.plot(iters, lh['train'], label='Train')
                ax.plot(iters, lh['val'],   label='Validation')
                ax.axvline(best + 1, color='red', ls='--', alpha=0.6, label=f'Best={best+1}')
                ax.set_title(f'Training Curve — {name} ({target_name})  ({lh["metric"]})')
                ax.set_xlabel('Itération'); ax.set_ylabel(lh['metric'])
                ax.legend(); fig.tight_layout()
                fig.savefig(output_dir / f'loss_curve_{target_name.replace(" ","_")}_{name.replace(" ","_")}.png', dpi=150)
                plt.show()

            results[name] = {
                'model': model, 'accuracy': acc, 'balanced_accuracy': bal_acc,
                'mcc': mcc, 'kappa': kappa, 'loss_history': loss_history,
                'confusion_matrix': cm,
                'report': classification_report(y_test, y_pred, target_names=label_names),
                'calibrated_model': calibrated_model,
                'calibrated_roc_auc': calibrated_roc,
                'optimal_threshold': opt_thr,
                'accuracy_opt': acc_opt,
                'balanced_accuracy_opt': bal_opt,
                'report_opt': report_opt,
            }

        except Exception as e:
            print(f'  ERREUR : {e}')
            import traceback; traceback.print_exc()

    return results


print('Helpers définis.')

In [ ]:
output_dir = Path(config.get_paths()['output_dir'])
output_dir.mkdir(parents=True, exist_ok=True)

home_results = _train_binary(
    make_models(), X_train_scaled, X_test_scaled, X_cal_scaled,
    y_home_train, y_home_test, y_home_cal,
    target_name='HomeWin',
    label_names=['Not HomeWin', 'HomeWin'],
    output_dir=output_dir
)
print('\nHomeWin — terminé.')

In [ ]:
# Courbes ROC — HomeWin (toutes les modèles)
fig, ax = plt.subplots(figsize=(10, 7))
for name, res in home_results.items():
    m = res['model']
    try:
        score = (m.predict_proba(X_test_scaled)[:, 1]
                 if hasattr(m, 'predict_proba')
                 else m.decision_function(X_test_scaled))
        fpr, tpr, _ = roc_curve(y_home_test, score)
        roc_auc_val = auc(fpr, tpr)
        res['roc_auc'] = roc_auc_val
        ax.plot(fpr, tpr, lw=2, label=f'{name} (AUC={roc_auc_val:.3f})')
    except Exception:
        pass

ax.plot([0,1],[0,1],'k--')
ax.set_xlabel('False Positive Rate'); ax.set_ylabel('True Positive Rate')
ax.set_title('ROC — HomeWin vs Not-HomeWin')
ax.legend(loc='lower right', fontsize=8)
fig.tight_layout()
fig.savefig(output_dir / 'roc_HomeWin.png', dpi=150)
plt.show()

In [ ]:
away_results = _train_binary(
    make_models(), X_train_scaled, X_test_scaled, X_cal_scaled,
    y_away_train, y_away_test, y_away_cal,
    target_name='AwayWin',
    label_names=['Not AwayWin', 'AwayWin'],
    output_dir=output_dir
)
print('\nAwayWin — terminé.')

In [ ]:
# Courbes ROC — AwayWin (toutes les modèles)
fig, ax = plt.subplots(figsize=(10, 7))
for name, res in away_results.items():
    m = res['model']
    try:
        score = (m.predict_proba(X_test_scaled)[:, 1]
                 if hasattr(m, 'predict_proba')
                 else m.decision_function(X_test_scaled))
        fpr, tpr, _ = roc_curve(y_away_test, score)
        roc_auc_val = auc(fpr, tpr)
        res['roc_auc'] = roc_auc_val
        ax.plot(fpr, tpr, lw=2, label=f'{name} (AUC={roc_auc_val:.3f})')
    except Exception:
        pass

ax.plot([0,1],[0,1],'k--')
ax.set_xlabel('False Positive Rate'); ax.set_ylabel('True Positive Rate')
ax.set_title('ROC — AwayWin vs Not-AwayWin')
ax.legend(loc='lower right', fontsize=8)
fig.tight_layout()
fig.savefig(output_dir / 'roc_AwayWin.png', dpi=150)
plt.show()

In [ ]:
def _summary_df(res_dict):
    rows = []
    for name, res in res_dict.items():
        rows.append({
            'Modèle':    name,
            'Accuracy':  res['accuracy'],
            'Bal. Acc':  res['balanced_accuracy'],
            'MCC':       res['mcc'],
            'Kappa':     res['kappa'],
            'ROC-AUC':   res.get('roc_auc', np.nan),
        })
    return pd.DataFrame(rows).sort_values('Accuracy', ascending=False)

home_df = _summary_df(home_results)
away_df = _summary_df(away_results)

pd.options.display.float_format = '{:.4f}'.format
print('=== HomeWin vs Not-HomeWin ===')
print(home_df.to_string(index=False))
print('\n=== AwayWin vs Not-AwayWin ===')
print(away_df.to_string(index=False))

# Graphiques comparatifs
for label, df in [('Home Win', home_df), ('Away Win', away_df)]:
    model_names = df['Modèle'].tolist()
    x, w = np.arange(len(model_names)), 0.28

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f'Comparaison modèles — {label}', fontsize=13)

    ax = axes[0]
    ax.bar(x - w/2, df['Accuracy'], w, label='Accuracy',     color='steelblue')
    ax.bar(x + w/2, df['Bal. Acc'], w, label='Balanced Acc', color='coral')
    ax.set_xticks(x); ax.set_xticklabels(model_names, rotation=30, ha='right')
    ax.set_ylim([0, 1]); ax.set_ylabel('Score'); ax.legend()
    ax.set_title('Accuracy vs Balanced Accuracy')

    ax = axes[1]
    ax.bar(x, df['ROC-AUC'].fillna(0), color='mediumseagreen')
    ax.axhline(0.5, color='red', ls='--', alpha=0.5, label='Random')
    ax.set_xticks(x); ax.set_xticklabels(model_names, rotation=30, ha='right')
    ax.set_ylim([0, 1]); ax.set_ylabel('ROC-AUC'); ax.legend()
    ax.set_title('ROC-AUC par modèle')

    fig.tight_layout()
    fig.savefig(output_dir / f'summary_comparison_{label.replace(" ","_")}.png', dpi=150)
    plt.show()

In [ ]:
metrics_file = output_dir / 'metrics_results_Binary.txt'
sorted_home = sorted(home_results.items(), key=lambda x: x[1]['accuracy'], reverse=True)
sorted_away = sorted(away_results.items(), key=lambda x: x[1]['accuracy'], reverse=True)

with open(metrics_file, 'w') as f:
    for target_label, sorted_res in [('HomeWin', sorted_home), ('AwayWin', sorted_away)]:
        f.write(f'\n{"="*70}\n=== {target_label} ===\n{"="*70}\n\n')

        # Ranking table
        f.write('=== RANKING TABLE ===\n')
        f.write(f"{'Modèle':<22} {'Accuracy':>9} {'Bal.Acc':>9} {'MCC':>8} {'Kappa':>8} {'ROC-AUC':>9}\n")
        f.write('-' * 70 + '\n')
        for name, res in sorted_res:
            roc_str = f"{res['roc_auc']:.4f}" if res.get('roc_auc') is not None else '  N/A  '
            f.write(f"{name:<22} {res['accuracy']:>9.4f} {res['balanced_accuracy']:>9.4f} "
                    f"{res['mcc']:>8.4f} {res['kappa']:>8.4f} {roc_str:>9}\n")
        f.write('\n')

        # Calibrated vs Uncalibrated AUC
        f.write('=== CALIBRATED vs UNCALIBRATED ROC-AUC ===\n')
        f.write(f"{'Modèle':<22} {'Uncalibrated':>14} {'Calibrated':>12} {'Delta':>8}\n")
        f.write('-' * 60 + '\n')
        for name, res in sorted_res:
            raw = res.get('roc_auc')
            cal = res.get('calibrated_roc_auc')
            raw_s = f"{raw:.4f}" if raw is not None else "   N/A"
            cal_s = f"{cal:.4f}" if cal is not None else "   N/A"
            delta_s = f"{cal - raw:+.4f}" if (raw is not None and cal is not None) else "   N/A"
            f.write(f"{name:<22} {raw_s:>14} {cal_s:>12} {delta_s:>8}\n")
        f.write('\n')

        # Optimised threshold
        f.write('=== OPTIMISED THRESHOLD METRICS ===\n')
        f.write(f"{'Modèle':<22} {'Acc(default)':>13} {'Acc(opt)':>10} {'BalAcc(default)':>16} {'BalAcc(opt)':>12}\n")
        f.write('-' * 77 + '\n')
        for name, res in sorted_res:
            acc_o_s = f"{res['accuracy_opt']:.4f}" if res.get('accuracy_opt') is not None else "   N/A"
            bal_o_s = f"{res['balanced_accuracy_opt']:.4f}" if res.get('balanced_accuracy_opt') is not None else "   N/A"
            f.write(f"{name:<22} {res['accuracy']:>13.4f} {acc_o_s:>10} "
                    f"{res['balanced_accuracy']:>16.4f} {bal_o_s:>12}\n")
        f.write('\n')

        # Detailed
        f.write('=== DETAILED RESULTS ===\n\n')
        for name, res in sorted_res:
            f.write(f'Modèle : {name}\n')
            f.write('-' * 30 + '\n')
            f.write(f"Accuracy          : {res['accuracy']:.4f}\n")
            f.write(f"Balanced Accuracy : {res['balanced_accuracy']:.4f}\n")
            f.write(f"MCC               : {res['mcc']:.4f}\n")
            f.write(f"Cohen's Kappa     : {res['kappa']:.4f}\n")
            if res.get('roc_auc') is not None:
                f.write(f"ROC-AUC           : {res['roc_auc']:.4f}\n")
            if res.get('calibrated_roc_auc') is not None:
                f.write(f"Calibrated ROC-AUC: {res['calibrated_roc_auc']:.4f}\n")
            if res.get('accuracy_opt') is not None:
                f.write(f"Accuracy (opt)    : {res['accuracy_opt']:.4f}\n")
                f.write(f"Bal.Acc  (opt)    : {res['balanced_accuracy_opt']:.4f}\n")
            f.write('\nClassification Report :\n')
            f.write(res['report'])
            if res.get('report_opt'):
                f.write('\nClassification Report (Optimised Threshold) :\n')
                f.write(res['report_opt'])
            if res.get('optimal_threshold') is not None:
                thr, f1 = res['optimal_threshold']
                f.write(f'\nOptimal Threshold : {thr:.4f}  (best F1={f1:.4f})\n')
            f.write('\nConfusion Matrix :\n')
            f.write(str(res['confusion_matrix']))
            lh = res.get('loss_history')
            if lh:
                _LOSS_METRICS = {'loss', 'logloss', 'multiclass', 'merror', 'mae', 'mse', 'rmse'}
                is_loss = any(tok in lh['metric'].lower() for tok in _LOSS_METRICS)
                best = int(np.argmin(lh['val']) if is_loss else np.argmax(lh['val']))
                f.write(f"\nTraining curve: metric={lh['metric']}, best_iter={best+1}, "
                        f"final_train={lh['train'][-1]:.4f}, final_val={lh['val'][-1]:.4f}\n")
            f.write('\n' + '=' * 70 + '\n')

print(f'Métriques sauvegardées : {metrics_file}')